In [1]:
!pip install transformers accelerate datasets torch torchvision peft pillow

In [2]:
from datasets import load_dataset, DatasetDict
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
Trainer,
TrainingArguments
)
import torch
from peft import (
LoraConfig,
get_peft_model,
TaskType,
PeftConfig,
PeftModel
)
from huggingface_hub import notebook_login

2025-10-10 15:00:45.314068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760108445.337488     189 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760108445.344336     189 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [4]:
ds = load_dataset("pourmand1376/persian-qa-translated")
ds

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'original_instruction', 'original_output', 'output', 'source'],
        num_rows: 153127
    })
})

In [5]:
ds = ds.remove_columns(["input", "original_instruction", "original_output", "source"])
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 153127
    })
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

def preprocess(sample):
    sample = sample["instruction"] + "\n" + sample["output"]
    tokenized = tokenizer(
        sample,
        max_length = 750,
        truncation = True,
        padding = "max_length"
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [7]:
ds_small = DatasetDict({
    "train": ds["train"].select(range(300))
})

In [8]:
ds_small

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 300
    })
})

In [9]:
data = ds_small.map(preprocess)
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    device_map = "auto",
    offload_folder = "offload",
    torch_dtype = torch.float16
)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
model.gradient_checkpointing_enable()

In [12]:
lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

In [13]:
model = get_peft_model(model, lora_config)

In [14]:
training_args = TrainingArguments(
    num_train_epochs = 25,
    learning_rate = 0.001,
    logging_steps = 25,
    #per_device_train_batch_size = 1,
    report_to = "tensorboard"
)

In [15]:
trainer = Trainer(
    model = model,
    train_dataset = data["train"],
    args = training_args
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
print("Start training...")
trainer.train()
print("Training finished...")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Start training...


Step,Training Loss
25,2.122300
50,1.189000
75,1.124700
100,1.096900
125,1.083300
150,1.054800
175,0.998400
200,1.005600
225,0.977100
250,0.891000


Training finished...


In [17]:
trainer.save_model("/kaggle/working/")
tokenizer.save_pretrained("/kaggle/working/")

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/chat_template.jinja',
 '/kaggle/working/vocab.json',
 '/kaggle/working/merges.txt',
 '/kaggle/working/added_tokens.json',
 '/kaggle/working/tokenizer.json')

In [18]:
path = "/kaggle/working/"

In [19]:
config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code = True)
model = PeftModel.from_pretrained(base, path)
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code = True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
def generate_response(query):
    inputs = tokenizer(query, return_tensors = "pt").to(model.device)
    output = model.generate(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        max_new_tokens = 512
    )
    return tokenizer.decode(output[0], skip_special_tokens = True)

In [21]:
print(generate_response("شرکت فولاد مبارکه در کجا واقع شده است"))

شرکت فولاد مبارکه در کجا واقع شده است؟
شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی خصوصی در ایران و بزرگ‌ترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم‌اکنون محرک بسیاری از صنایع بالادستی و پایین‌دستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده‌است و همچنین این شرکت در سال ۱۳۹۱ برای نخستین‌بار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ‌ترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده‌است. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده‌رود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم‌آبی زاینده‌رود را تحریف می‌ک


In [22]:
print(generate_response("تعریف علوم کامپیوترچیست؟"))

تعریف علوم کامپیوترچیست؟
علوم رایانه یا علوم کامپیوتر به مجموعهٔ مطالعاتی گفته می‌شود که به زیربناهای نظری، روش‌های طراحی و ساخت و چگونگی استفاده از رایانه می‌پردازند.رشته علوم کامپیوتر را می‌توان به زیررشته‌های نظری و عملی بسیاری تقسیم کرد. بعضی از این زیررشته‌ها، نظیر نظریه پیچیدگی محاسباتی (که خواص اساسی مشکلات محاسباتی و قابل حل بودن آن‌ها را بررسی می‌کند) بسیار انتزاعی هستند، این در حالی است که زیررشته‌های دیگر مانند گرافیک کامپیوتری به بررسی کاربردهای قابل لمس تر در دنیای واقعی تأکید دارند. اکثر زیررشته‌های علوم کامپیوتر بر چالش‌های موجود در اجرای محاسبات تمرکز دارند.  اولین مؤسسهٔ علمی که عبارت داده‌شناسی را بکار برد DIKU گروه داده‌شناسی در دانشگاه کپنهاگ بوده‌است که توسط پیتر ناور در سال ۱۹۶۹ به عنوان اولین گروه داده‌شناسی بنا گذاشته شد. عبارت زیر از دانشمند معروف علم رایانه ادسخر دیکسترا نقل قول شده‌است: "علم رایانه به همان اندازه در مورد رایانه است که نجوم در مورد تلسکوپ." در بسیاری از دانشگاه‌ها و دانشک
